In [1]:
from tqdm import tqdm


### Filter ES ticks into a single continous contract

In [3]:
csv = "../../../../Historical Data/ES-Futures-Ticks-20230807-20240806.trades.csv" # input
filtered_csv = "../../../../Historical Data/ES-Futures-Ticks-20230807-20240806.trades.filtered.csv" # output

with open(csv, "r") as f:
    for i in f:
        print("Column headers: \n", "\n".join(
            [str(x) for x in enumerate(i.split(","))]))
        break

Column headers: 
 (0, 'ts_recv')
(1, 'ts_event')
(2, 'rtype')
(3, 'publisher_id')
(4, 'instrument_id')
(5, 'action')
(6, 'side')
(7, 'depth')
(8, 'price')
(9, 'size')
(10, 'flags')
(11, 'ts_in_delta')
(12, 'sequence')
(13, 'symbol\n')


In [4]:
"""
We want ts_event as time, price, size as volume, symbol
So we want column numbers 1, 8, 9, 13
The symbols we want are ESU3, ESZ3, ESH4, ESM4, ESU4
"""
data = {"time" : [], "price" : [], "volume" : [], "symbol" : []}
with open(csv, "r") as f:
    for i, line in enumerate(tqdm(f, total=106615135)):
        if i == 0:
            continue
        line = line.strip().split(",")
        if line[13] not in ["ESU3", "ESZ3", "ESH4", "ESM4", "ESU4"]:
            continue
        data["time"].append(line[1])
        data["price"].append(line[8])
        data["volume"].append(line[9])
        data["symbol"].append(line[13])

  0%|          | 0/106615135 [00:00<?, ?it/s]

100%|██████████| 106615135/106615135 [01:10<00:00, 1503477.46it/s]


In [5]:
"""
We start to see some trades of ESZ3 appearing during the ESU3 contract
Let's further filter the data so that each contract reaches its end date
prior to admitting prices from the next contract
To do this, we need to identify the end index of each contract
"""
esu3_end = 0
esz3_end = 0
esh4_end = 0
esm4_end = 0

for i, s in enumerate(tqdm(data["symbol"])):
    if s == "ESU3":
        esu3_end = i
    elif s == "ESZ3":
        esz3_end = i
    elif s == "ESH4":
        esh4_end = i
    elif s == "ESM4":
        esm4_end = i

print(esu3_end, esz3_end, esh4_end, esm4_end)

  0%|          | 0/105736599 [00:00<?, ?it/s]

100%|██████████| 105736599/105736599 [01:09<00:00, 1525311.72it/s]

12304220 41704811 64404499 89984159


In [6]:
print(f"""
ESU3: {data["time"][0]} - {data["time"][esu3_end]}
ESZ3: {data["time"][esu3_end]} - {data["time"][esz3_end]}
ESH4: {data["time"][esz3_end]} - {data["time"][esh4_end]}
ESM4: {data["time"][esh4_end]} - {data["time"][esm4_end]}
      """.strip())

ESU3: 2023-08-07T00:00:00.039623665Z - 2023-09-15T13:29:59.452410495Z
ESZ3: 2023-09-15T13:29:59.452410495Z - 2023-12-15T14:29:59.410358005Z
ESH4: 2023-12-15T14:29:59.410358005Z - 2024-03-15T13:29:59.847946481Z
ESM4: 2024-03-15T13:29:59.847946481Z - 2024-06-21T13:29:59.522321675Z


In [7]:
filtered_data = {"time" : [], "price" : [], "volume" : [], "symbol" : []}
for i in tqdm(range(len(data["symbol"]))):
    if (i <= esu3_end and data["symbol"][i] == "ESU3") or \
       (i > esu3_end and i <= esz3_end and data["symbol"][i] == "ESZ3") or \
       (i > esz3_end and i <= esh4_end and data["symbol"][i] == "ESH4") or \
       (i > esh4_end and i <= esm4_end and data["symbol"][i] == "ESM4"):
        filtered_data["time"].append(data["time"][i])
        filtered_data["price"].append(data["price"][i])
        filtered_data["volume"].append(data["volume"][i])
        filtered_data["symbol"].append(data["symbol"][i])


100%|██████████| 105736599/105736599 [01:20<00:00, 1309709.43it/s]


In [8]:
len(data["symbol"]), len(filtered_data["symbol"])

(105736599, 82971690)

In [9]:
with open (filtered_csv, "w") as f:
    f.write("time,price,volume,symbol\n")
    for i in tqdm(range(len(filtered_data["symbol"]))):
        f.write(f"{filtered_data['time'][i]},{filtered_data['price'][i]},{filtered_data['volume'][i]},{filtered_data['symbol'][i]}\n")

100%|██████████| 82971690/82971690 [01:26<00:00, 958235.74it/s] 


### Once the above data is filtered, we can read it in on subsequent runs. 

In [1]:
from tqdm import tqdm
import pandas as pd
import sys; sys.path.append("..")
from bars.imbalance_bars import tick_imbalance_bars, volume_imbalance_bars
# from utils.visualizations.plot_bars_on_prices import plot_bars_on_prices
import plotly.graph_objects as go

In [2]:
filtered_csv = "../../../../Historical Data/ES-Futures-Ticks-20230807-20240806.trades.filtered.csv" # now input

data = {"time" : [], "price" : [], "volume" : [], "symbol" : []}
with open(filtered_csv, "r") as f:
    for i, line in enumerate(tqdm(f, total=82971690)):
        if i == 0:
            continue
        line = line.strip().split(",")
        data["time"].append(line[0])
        data["price"].append(float(line[1]))
        data["volume"].append(float(line[2]))
        data["symbol"].append(line[3])

82971691it [00:45, 1822065.13it/s]                              


In [3]:
bars = volume_imbalance_bars(data, alpha=0.001, et_init=500, verbose=True)

  0%|          | 251380/82971689 [00:00<01:03, 1308211.95it/s]

Making a new bar from 0 to 16
Making a new bar from 16 to 85
Making a new bar from 85 to 102
Making a new bar from 102 to 114
Making a new bar from 114 to 155
Making a new bar from 155 to 237
Making a new bar from 237 to 257
Making a new bar from 257 to 287
Making a new bar from 287 to 367
Making a new bar from 367 to 425
Making a new bar from 425 to 449
Making a new bar from 449 to 462
Making a new bar from 462 to 488
Making a new bar from 488 to 514
Making a new bar from 514 to 614
Making a new bar from 614 to 625
Making a new bar from 625 to 653
Making a new bar from 653 to 746
Making a new bar from 746 to 786
Making a new bar from 786 to 813
Making a new bar from 813 to 880
Making a new bar from 880 to 907
Making a new bar from 907 to 933
Making a new bar from 933 to 979
Making a new bar from 979 to 1016
Making a new bar from 1016 to 1049
Making a new bar from 1049 to 1170
Making a new bar from 1170 to 1180
Making a new bar from 1180 to 1292
Making a new bar from 1292 to 1340
Makin

  1%|          | 604585/82971689 [00:00<00:58, 1409344.14it/s]

Making a new bar from 360208 to 362749
Making a new bar from 362749 to 367979
Making a new bar from 367979 to 373618
Making a new bar from 373618 to 374018
Making a new bar from 374018 to 374839
Making a new bar from 374839 to 375088
Making a new bar from 375088 to 376722
Making a new bar from 376722 to 377505
Making a new bar from 377505 to 383348
Making a new bar from 383348 to 428257
Making a new bar from 428257 to 449221
Making a new bar from 449221 to 473516
Making a new bar from 473516 to 483669
Making a new bar from 483669 to 496528
Making a new bar from 496528 to 500456
Making a new bar from 500456 to 508983
Making a new bar from 508983 to 510003
Making a new bar from 510003 to 513009
Making a new bar from 513009 to 514017
Making a new bar from 514017 to 514099
Making a new bar from 514099 to 514358
Making a new bar from 514358 to 514531
Making a new bar from 514531 to 514653
Making a new bar from 514653 to 514797
Making a new bar from 514797 to 514825
Making a new bar from 514

  1%|          | 887289/82971689 [00:00<01:02, 1303073.26it/s]

...

  5%|▌         | 4154385/82971689 [00:02<00:43, 1801779.72it/s]

Making a new bar from 3415557 to 3796490


 13%|█▎        | 10629197/82971689 [00:06<00:40, 1775489.01it/s]

Making a new bar from 3796490 to 10708698


100%|██████████| 82971689/82971689 [00:50<00:00, 1652513.32it/s]


In [4]:
len(bars['start_idx'])

6439

In [1]:
# use plotly to plot the bars as candles
fig = go.Figure(data=[go.Candlestick(x=[i for i in range(len(bars['start_idx']))],
                open=bars['open'],
                high=bars['high'],
                low=bars['low'],
                close=bars['close'])])

fig.show()

NameError: name 'go' is not defined